# Competición: predicción precios ordenadores (TC: Kaggle)

## Importación librerías

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pprint import *

import re


from EDA_function_tools import *
from toolbox_DS import *


## Carga datos y primer vistazo

In [2]:
df = pd.read_csv('./data/test.csv', index_col=0)
df.head(8)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
id,,,,,,,,,,,,
181,1098,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg
708,330,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8GB,256GB SSD,AMD Radeon RX 540,Windows 10,2.2kg
862,1260,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,500GB HDD,Intel HD Graphics 520,Linux,2.4kg
1064,1137,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.43kg
702,1015,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.34kg
585,1220,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,512GB SSD,Intel UHD Graphics 620,Windows 10,2kg
195,787,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg
463,841,Asus,VivoBook Max,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,1TB HDD,Nvidia GeForce 920,Linux,2.1kg


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         391 non-null    int64  
 1   Company           391 non-null    object 
 2   Product           391 non-null    object 
 3   TypeName          391 non-null    object 
 4   Inches            391 non-null    float64
 5   ScreenResolution  391 non-null    object 
 6   Cpu               391 non-null    object 
 7   Ram               391 non-null    object 
 8   Memory            391 non-null    object 
 9   Gpu               391 non-null    object 
 10  OpSys             391 non-null    object 
 11  Weight            391 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 39.7+ KB


## Limpieza y transformación de variables

### Duplicados y nulos

#### Duplicados

In [4]:
df.duplicated().sum()

0

No hay duplicados

#### Nulos

In [5]:
df.isna().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
dtype: int64

No hay nulos

Primeras conclusiones:
- ~~Eliminar la columna laptop_ID~~
- ~~Eliminar el nombre del producto Product~~
- ~~ Modificar la columna ScreenResolution sacando solo la resolución de primeras~~
    - ~~Habría que valorar también sacar: HD, pantalla táctil, IPS Panel, 4K, ultra~~
- ~~Modificar columna Cpu sacando diferentes componentes como:~~
    - ~~Inte Core~~
    - ~~Nombre~~
    - ~~Velocidad en ghz~~
- ~~Eliminar de la columna Ram las letras GB~~
- ~~La feature Memory no sé muy bien cómo afrontarla... pero algo hay que hacer con ella~~
- ~~Modificar la feature Gpu sacando por un lado el nombre y por otro el modelo? o dejrala como está de momento~~
- ~~Eliminar de la variable Weight las letras kg~~

Supuestametne habría que hacer primero la modificación en el train_set y con los valores de éste la modificación en el test_set, pero como es el set de train, lo hago todo ahora y luego hago la división.  
Esto mismo se hará al set específico de test.

### Tratamiento de variables

In [6]:
# Hago copia del df y elimino las dos columnas comantadas 'laptot_ID' y 'Product'
df_copy = df.copy()
df_copy = df_copy.drop(['laptop_ID','Product'],axis = 1)

#### Variable Ram

In [7]:
# Elimino las letras 'GB' de la varialbe 'Ram'
df_copy['Ram'] = df_copy['Ram'].str.replace('GB','')

#### Variable Weight

In [8]:
# Elimino las letras 'kg' de la variable 'Weight'
df_copy['Weight'] = df_copy['Weight'].str.replace('kg','')

In [9]:
# Reviso los cambios realizados
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
id,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.3
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.2
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.4
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.1


In [10]:
# Veo si se han modificado los tipos de las variables 'Ram' y 'Weight'
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   TypeName          391 non-null    object 
 2   Inches            391 non-null    float64
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Ram               391 non-null    object 
 6   Memory            391 non-null    object 
 7   Gpu               391 non-null    object 
 8   OpSys             391 non-null    object 
 9   Weight            391 non-null    object 
dtypes: float64(1), object(9)
memory usage: 33.6+ KB


In [11]:
# Cambio el tipo de varialbe a 'float' de 'Ram' y 'Weight'
df_copy['Ram'] = df_copy['Ram'].astype('int')
df_copy['Weight'] = df_copy['Weight'].astype('float')

In [12]:
# Compruebo el cambio
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   TypeName          391 non-null    object 
 2   Inches            391 non-null    float64
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Ram               391 non-null    int32  
 6   Memory            391 non-null    object 
 7   Gpu               391 non-null    object 
 8   OpSys             391 non-null    object 
 9   Weight            391 non-null    float64
dtypes: float64(2), int32(1), object(7)
memory usage: 32.1+ KB


#### Variable ScreenResolution

In [14]:
# Obtengo la resolución en valores 
resolution = []
for resolucion in df_copy['ScreenResolution']:
    x = re.findall('\d\d\d\dx\d\d\d?\d',resolucion)
    resolution.append(x)
print(resolution)

[['3840x2160'], ['1366x768'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2560x1440'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2560x1440'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2736x1824'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1366x768'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1366x768'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2160x1440'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2256x1504'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x

In [15]:
# Obtengo una lista única (dado que tenía una lista de listas)
lista_resolution = []
for n in range(0, len(resolution)):
    valor = resolution[n][0]
    lista_resolution.append(valor)
print(lista_resolution)


['3840x2160', '1366x768', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2560x1440', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2560x1440', '1920x1080', '1920x1080', '1920x1080', '3200x1800', '1920x1080', '1920x1080', '1920x1080', '2736x1824', '1920x1080', '1366x768', '1920x1080', '1366x768', '3200x1800', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1366x768', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2160x1440', '1920x1080', '1920x1080', '1366x768', '3200x1800', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '2256x1504', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1600x900', '1920x1080', '1600x900', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1366x768', '

In [16]:
# Añado nueva variable con la información
df_copy['Resolution'] = lista_resolution

In [17]:
# Compruebo
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution
id,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080


In [18]:
tipos_pantalla =[]
screen_types = {'Pantalla':['4K Ultra HD', 'Full HD', 'Quad HD+', 'Retina Display','Touchscreen']}

for n in df_copy['ScreenResolution']:
    found = False  
    for p in screen_types['Pantalla']:
        if p in n:
            tipos_pantalla.append(p)
            found = True
            break  
    if not found:
        tipos_pantalla.append('Full HD')

print(tipos_pantalla)


['4K Ultra HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Retina Display', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', '

In [19]:
df_copy['Pantalla'] = tipos_pantalla

In [20]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Pantalla
id,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,4K Ultra HD
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,Full HD
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,Full HD
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,Full HD
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,Full HD
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080,Full HD
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080,Full HD
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080,Full HD


#### Variable Cpu

In [21]:
# Función para extraer el fabricante (brand)
def extract_brand(cpu):
    return cpu.split()[0]

# Función para extraer la familia (family)
def extract_family(cpu):
    parts = cpu.split()
    if parts[1] == 'Core':
        return parts[1] + ' ' + parts[2]
    else:
        return parts[1]

# Función para extraer el modelo
def extract_model(cpu):
    parts = cpu.split()
    if parts[1] == 'Core':
        return parts[3]
    else:
        return parts[2]

# Función para extraer la velocidad de reloj (clock speed)
def extract_clock_speed(cpu):
    return float(cpu.split()[-1].replace('GHz', ''))

# Aplicar las funciones para crear nuevas columnas
df_copy['Brand'] = df_copy['Cpu'].apply(extract_brand)
df_copy['Family'] = df_copy['Cpu'].apply(extract_family)
df_copy['Model'] = df_copy['Cpu'].apply(extract_model)
df_copy['Clock Speed (GHz)'] = df_copy['Cpu'].apply(extract_clock_speed)

In [22]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Pantalla,Brand,Family,Model,Clock Speed (GHz)
id,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,4K Ultra HD,Intel,Core i7,7500U,2.7
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,Full HD,AMD,A12-Series,9720P,2.7
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,Full HD,Intel,Core i3,6006U,2.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,Full HD,Intel,Core i5,6200U,2.3
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,Full HD,Intel,Core i5,7200U,2.5
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080,Full HD,Intel,Core i7,8550U,1.8
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080,Full HD,Intel,Core i7,7700HQ,2.8
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080,Full HD,Intel,Core i5,7200U,2.5


In [23]:
# La varaible Model tiene demasiados elementos para el one-hot-encoding. 
# La voy a discretizar en base al rendimiento esperado de los procesadores
cpu_categories = {
    '7500U': 'Media', 'Quad': 'Alta', '6200U': 'Media', '7700HQ': 'Alta', '6300U': 'Media',
    '6006U': 'Baja', '8550U': 'Alta', 'A9-9420': 'Baja', 'Dual': 'Baja', '8250U': 'Alta',
    '7100U': 'Media', '7200U': 'Media', 'E3-1535M': 'Alta', 'x5-Z8550': 'Baja', '9620P': 'Media',
    '7300U': 'Media', '6500U': 'Media', '9720P': 'Media', '7130U': 'Media', '7300HQ': 'Alta',
    '7820HQ': 'Alta', '2.2GHz': 'Media', '6600U': 'Media', 'x5-Z8350': 'Baja', '6Y75': 'Baja',
    '7600U': 'Media', '7820HK': 'Alta', 'E2-9000': 'Baja', '9420': 'Baja', '8800P': 'Media',
    'A10-9620P': 'Media', '6700HQ': 'Alta', '7Y30': 'Baja', '7Y75': 'Baja', '9220': 'Baja',
    '6300HQ': 'Media', '1600': 'Alta', 'E2-6110': 'Baja', '6110': 'Baja', '9410': 'Baja',
    '7440HQ': 'Alta', '1.3GHz': 'Baja', '1.8GHz': 'Baja', '6820HK': 'Alta', 'M3-6Y30': 'Baja',
    '6820HQ': 'Alta', '6100U': 'Baja', '3.1GHz': 'Alta', '6Y30': 'Baja', '1.2GHz': 'Baja',
    '7410': 'Baja', '7Y54': 'Baja', '8650U': 'Alta', '2.3GHz': 'Media', '7660U': 'Media',
    '7Y57': 'Baja', '2.8GHz': 'Alta', 'E3-1505M': 'Alta', '1700': 'Alta', '1.1GHz': 'Baja',
    '7110': 'Baja', '9000e': 'Baja', 'M7-6Y75': 'Baja', 'm3': 'Baja', 'X5-Z8350': 'Baja',
    '2.7GHz': 'Alta', '9830P': 'Media', 'm3-7Y30': 'Baja', '2.9GHz': 'Alta', '6920HQ': 'Alta',
    '7560U': 'Media', '1.6GHz': 'Baja', '6560U': 'Media', 'x5-Z8300': 'Baja', 'Z8350': 'Baja',
    '9700P': 'Media', 'A6-9220': 'Baja', '9000': 'Baja'
}

# Crear una nueva columna 'CPU_Category' basada en el diccionario cpu_categories
df_copy['CPU_Model_disc'] = df_copy['Model'].map(cpu_categories)

# Manejar CPUs que no están en el diccionario (opcional)
df_copy['CPU_Model_disc'] = df_copy['CPU_Model_disc'].fillna('Desconocida')


#### Variable Gpu

In [24]:
# Función para extraer el fabricante (brand)
def extract_gpu_brand(gpu):
    return gpu.split()[0]

# Función para extraer el modelo
def extract_gpu_model(gpu):
    return ' '.join(gpu.split()[1:])

# Crear una lista de GPUs integradas y dedicadas para clasificar el tipo
integrated_gpus = ['Intel HD Graphics', 'Intel UHD Graphics', 'Intel Iris Plus Graphics', 'Intel Iris Graphics', 'Intel HD Graphics 5000', 'Intel HD Graphics 6000', 'Intel HD Graphics 5200', 'Intel Iris Pro Graphics', 'Intel Iris Graphics', 'Intel UHD Graphics']
dedicated_gpus = ['Nvidia GeForce', 'Nvidia Quadro', 'AMD Radeon', 'AMD FirePro']

# Función para clasificar el tipo de GPU
def classify_gpu_type(gpu):
    brand_model = ' '.join(gpu.split()[:2])
    if any(integrated in brand_model for integrated in integrated_gpus):
        return 'Integrated'
    elif any(dedicated in brand_model for dedicated in dedicated_gpus):
        return 'Dedicated'
    else:
        return 'Unknown'

# Aplicar las funciones para crear nuevas columnas
df_copy['Gpu Brand'] = df_copy['Gpu'].apply(extract_gpu_brand)
df_copy['Gpu Model'] = df_copy['Gpu'].apply(extract_gpu_model)
df_copy['Gpu Type'] = df_copy['Gpu'].apply(classify_gpu_type)

In [25]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Pantalla,Brand,Family,Model,Clock Speed (GHz),CPU_Model_disc,Gpu Brand,Gpu Model,Gpu Type
id,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,4K Ultra HD,Intel,Core i7,7500U,2.7,Media,Intel,HD Graphics 620,Unknown
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,Full HD,AMD,A12-Series,9720P,2.7,Media,AMD,Radeon RX 540,Dedicated
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,Full HD,Intel,Core i3,6006U,2.0,Baja,Intel,HD Graphics 520,Unknown
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,Full HD,Intel,Core i5,6200U,2.3,Media,Intel,HD Graphics 520,Unknown
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,Full HD,Intel,Core i5,7200U,2.5,Media,Intel,HD Graphics 620,Unknown
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080,Full HD,Intel,Core i7,8550U,1.8,Alta,Intel,UHD Graphics 620,Unknown
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080,Full HD,Intel,Core i7,7700HQ,2.8,Alta,Nvidia,GeForce GTX 1060,Dedicated
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080,Full HD,Intel,Core i5,7200U,2.5,Media,Nvidia,GeForce 920,Dedicated


In [26]:
# Los valores de la columna Gpu Model son demasiados para hacer un one-hot-encoding.
# Hago una clasificación en base a su rendimiento estimado

gpu_categories = {
    'Radeon R7 M445': 'Media', 'HD Graphics 505': 'Baja', 'HD Graphics 520': 'Baja',
    'GeForce GTX 1070': 'Alta', 'GeForce GT 940MX': 'Media', 'GeForce GTX 1050': 'Media',
    'GeForce 940MX': 'Media', 'GeForce GTX 1050 Ti': 'Media', 'HD Graphics 405': 'Baja',
    'UHD Graphics 620': 'Baja', 'Radeon 530': 'Baja', 'HD Graphics 510': 'Baja',
    'HD Graphics 620': 'Baja', 'Quadro M2200M': 'Alta', 'GeForce MX150': 'Media',
    'HD Graphics 400': 'Baja', 'Quadro M1200': 'Alta', 'GeForce 920MX': 'Baja',
    'HD Graphics 500': 'Baja', 'Radeon 520': 'Baja', 'Radeon R5 M330': 'Baja',
    'Radeon R5 M420X': 'Baja', 'HD Graphics': 'Baja', 'HD Graphics 630': 'Baja',
    'Radeon R5 M430': 'Baja', 'Iris Pro Graphics': 'Media', 'GeForce GTX 940MX': 'Media',
    'HD Graphics 515': 'Baja', 'GeForce GTX 1080': 'Alta', 'Radeon R2 Graphics': 'Baja',
    'GeForce 150MX': 'Media', 'Radeon R5': 'Baja', 'GeForce GTX 1050Ti': 'Media',
    'Radeon R9 M385': 'Alta', 'GeForce 930MX': 'Baja', 'GeForce GTX 1060': 'Alta',
    'GeForce GTX 950M': 'Media', 'GeForce GTX 960M': 'Media', 'HD Graphics 615': 'Baja',
    'FirePro W4190M': 'Alta', 'Radeon R4': 'Baja', 'Quadro M500M': 'Alta',
    'Radeon RX 580': 'Alta', 'GeForce GTX 970M': 'Alta', 'Radeon R7 M365X': 'Media',
    'Radeon R2': 'Baja', 'Quadro M1000M': 'Alta', 'GeForce GTX 965M': 'Alta',
    'GeForce MX130': 'Baja', 'Radeon R5 M420': 'Baja', 'Radeon R7 M440': 'Media',
    'HD Graphics 6000': 'Baja', 'GeForce GTX 980M': 'Alta', 'Quadro M2000M': 'Alta',
    'Iris Plus Graphics 650': 'Media', 'Radeon RX 550': 'Media', 'Iris Plus Graphics 640': 'Media',
    'HD Graphics 5300': 'Baja', 'Radeon R5 520': 'Baja', 'GeForce 960M': 'Media',
    'GeForce 930M': 'Baja', 'Quadro 3000M': 'Alta', 'GeForce GTX 1070M': 'Alta',
    'Quadro M620M': 'Alta', 'Radeon R5 M315': 'Baja', 'Radeon Pro 555': 'Alta',
    'Quadro M620': 'Alta', 'GeForce GTX 960<U+039C>': 'Media', 'Quadro M2200': 'Alta',
    'GeForce 920M': 'Baja', 'Quadro M3000M': 'Alta', 'R17M-M1-70': 'Media',
    'GeForce GTX 940M': 'Media', 'Radeon R7 M460': 'Media', 'Radeon R7 M360': 'Media',
    'Radeon R4 Graphics': 'Baja', 'Radeon Pro 455': 'Alta', 'Radeon RX 560': 'Media',
    'Iris Graphics 550': 'Media', 'GTX 980 SLI': 'Alta', 'R4 Graphics': 'Baja',
    'GeForce GTX1050 Ti': 'Media', 'Iris Graphics 540': 'Media', 'FirePro W6150M': 'Alta',
    'Radeon R7 Graphics': 'Baja', 'Radeon R7': 'Baja', 'Graphics 620': 'Baja', 'Quadro M520M': 'Alta'
}

df_copy['GPU_Model_disc'] = df_copy['Gpu Model'].map(gpu_categories)

#### Variable Memory

In [27]:
# Función para extraer el tipo de almacenamiento (Storage Type)
def extract_storage_type(memory):
    if 'SSD' in memory:
        return 'SSD'
    elif 'HDD' in memory:
        return 'HDD'
    elif 'Hybrid' in memory:
        return 'Hybrid'
    elif 'Flash Storage' in memory:
        return 'Flash Storage'
    else:
        return 'Unknown'

# Función para extraer la capacidad de almacenamiento (Storage Capacity)
def extract_storage_capacity(memory):
    # Buscar el primer número seguido opcionalmente por 'GB' o 'TB'
    match = re.search(r'(\d+)(?:GB|TB)', memory)
    if match:
        # Extraer el número encontrado en el grupo de captura 1
        number = int(match.group(1))
        medida = match.group(0)[-2:]
        # Verificar si hay 'TB' o 'GB' en la cadena y ajustar la capacidad
        if 'TB' in medida:
            number *= 1000
        return number
    return None

# Aplicar las funciones para crear nuevas columnas
df_copy['Storage Type'] = df_copy['Memory'].apply(extract_storage_type)
df_copy['Storage Capacity (GB)'] = df_copy['Memory'].apply(extract_storage_capacity)

In [28]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Family,Model,Clock Speed (GHz),CPU_Model_disc,Gpu Brand,Gpu Model,Gpu Type,GPU_Model_disc,Storage Type,Storage Capacity (GB)
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,Core i7,7500U,2.7,Media,Intel,HD Graphics 620,Unknown,Baja,SSD,512
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,A12-Series,9720P,2.7,Media,AMD,Radeon RX 540,Dedicated,NaN,SSD,256
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,Core i3,6006U,2.0,Baja,Intel,HD Graphics 520,Unknown,Baja,HDD,500
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,Core i5,6200U,2.3,Media,Intel,HD Graphics 520,Unknown,Baja,SSD,256
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,Core i5,7200U,2.5,Media,Intel,HD Graphics 620,Unknown,Baja,SSD,256
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,...,Core i7,8550U,1.8,Alta,Intel,UHD Graphics 620,Unknown,Baja,SSD,512
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,...,Core i7,7700HQ,2.8,Alta,Nvidia,GeForce GTX 1060,Dedicated,Alta,SSD,512
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,...,Core i5,7200U,2.5,Media,Nvidia,GeForce 920,Dedicated,NaN,HDD,1000


### DataFrame final

In [29]:
df_copy.columns

Index(['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram',
       'Memory', 'Gpu', 'OpSys', 'Weight', 'Resolution', 'Pantalla', 'Brand',
       'Family', 'Model', 'Clock Speed (GHz)', 'CPU_Model_disc', 'Gpu Brand',
       'Gpu Model', 'Gpu Type', 'GPU_Model_disc', 'Storage Type',
       'Storage Capacity (GB)'],
      dtype='object')

In [30]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Company                391 non-null    object 
 1   TypeName               391 non-null    object 
 2   Inches                 391 non-null    float64
 3   ScreenResolution       391 non-null    object 
 4   Cpu                    391 non-null    object 
 5   Ram                    391 non-null    int32  
 6   Memory                 391 non-null    object 
 7   Gpu                    391 non-null    object 
 8   OpSys                  391 non-null    object 
 9   Weight                 391 non-null    float64
 10  Resolution             391 non-null    object 
 11  Pantalla               391 non-null    object 
 12  Brand                  391 non-null    object 
 13  Family                 391 non-null    object 
 14  Model                  391 non-null    object 
 15  Clock Sp

### Exportación del df a un nuevo archivo

In [31]:
df_copy.to_csv('./data/df_precios_ordenadores_test.csv')